In [51]:
import pandas as pd
import numpy as np

df = pd.read_csv("s&p500ret.csv")

# 列名去空格
df.columns = df.columns.str.strip()

# 类型转换
df["date"] = pd.to_datetime(df["date"])
df["RET"] = pd.to_numeric(df["RET"], errors="coerce")

df = df.sort_values(["PERMNO", "date"])

# 映射到该周周五
df["week_end"] = df["date"] + pd.offsets.Week(weekday=4)

# 1. 周收益：每只股票、每周的复利收益
week_ret = (
    df
    .groupby(["PERMNO", "week_end"])["RET"]
    .apply(lambda r: (1 + r).prod() - 1)
    .reset_index(name="RET_w")
)

# 2. 12周动量（不含本周）
week_ret = week_ret.sort_values(["PERMNO", "week_end"])
g = week_ret.groupby("PERMNO")["RET_w"]

week_ret["MOM_12w"] = (
    (1 + g.shift())        # 往前挪一周，避免用到本周
    .rolling(window=12, min_periods=12)
    .apply(np.prod, raw=True) - 1
)

print(week_ret.head())


/var/folders/pv/7p_t7rb926b5n_ct8w7y94v80000gn/T/ipykernel_64352/2527902592.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("s&p500ret.csv")


   PERMNO   week_end     RET_w  MOM_12w
0   10104 2019-01-04 -0.008195      NaN
1   10104 2019-01-11  0.067887      NaN
2   10104 2019-01-18  0.019884      NaN
3   10104 2019-01-25  0.012762      NaN
4   10104 2019-02-01  0.020935      NaN


In [52]:
week_ret = week_ret.rename(columns = {"PERMNO":"TICKER", "week_end":"week_date"})
week_ret['week_date'] = (
    week_ret['week_date']
        .dt.to_period('W-FRI')
        .dt.to_timestamp(how='end')   # 取周期的结束日 = 周五
        .dt.normalize()
)
from pandas import Timedelta
week_ret["week_date"] = week_ret["week_date"] + Timedelta(days=1)
week_ret

,TICKER,week_date,RET_w,MOM_12w
0,10104,2019-01-05,-0.008195,NaN
1,10104,2019-01-12,0.067887,NaN
2,10104,2019-01-19,0.019884,NaN
3,10104,2019-01-26,0.012762,NaN
4,10104,2019-02-02,0.020935,NaN
...,...,...,...,...
155149,93436,2024-12-07,0.109947,0.446280
155150,93436,2024-12-14,0.131558,0.607809
155151,93436,2024-12-21,0.043220,0.714087
155152,93436,2024-12-28,0.041177,0.715720


In [55]:
week_ret = week_ret[week_ret["week_date"] >= pd.Timestamp("2020-01-01")]
week_ret = week_ret[["TICKER","week_date","MOM_12w"]]
# week_ret = week_ret.rename(columns = {"PERMNO":"TICKER", "week_end":"week_date"})
# week_ret['week_date']= week_ret['week_date'].dt.to_period('W-FRI').dt.to_timestamp()
week_ret.to_csv('../../data/weekly_momentum.csv', index=False)

In [54]:
week_ret

,TICKER,week_date,MOM_12w
52,10104,2020-01-04,-0.001008
53,10104,2020-01-11,-0.027753
54,10104,2020-01-18,-0.007973
55,10104,2020-01-25,0.026091
56,10104,2020-02-01,0.013466
...,...,...,...
155149,93436,2024-12-07,0.446280
155150,93436,2024-12-14,0.607809
155151,93436,2024-12-21,0.714087
155152,93436,2024-12-28,0.715720


In [26]:
import os
print("cwd =", os.getcwd())
print(".. 里面有什么：", os.listdir("../.."))
print("../data 是否存在：", Path("../../data").exists())

cwd = /Users/lizhu/Desktop/ML-Capstone-Project-006/code/momentum
.. 里面有什么： ['literature', 'requirements.txt', 'code', 'docs', 'README.md', 'FinEntity', '.gitignore', 'CONTRIBUTING.md', '.venv', '.gitattributes', '.git', '.vscode', 'data']
../data 是否存在： True


In [ ]:
out